# CoilcraftLTLibProcessor
## Purpose
Process Coilcraft LTSpice library file (.lib) into a CSV file for easier manipulation by Python scripts.

In [13]:
from pathlib import Path
import re
import pandas as pd

## Process Function
This function takes in a single LTSpice .lib file name (relative to the current directory) and converts it into a list of dicts which contains the inductance, part number, and subcircuit (subckt) data for each model in the library. *Note that this is not a general purpose function and will only work with Coilcraft library data in the following example format:*

```
*======================================================================
* SPICE Model generated by Coilcraft
* Coilcraft Part Number : XGL1010-682
* Inductance = 6.8uH
*======================================================================
* Model Parameters:
*   Valid Frequency Range = 0.1MHz-50MHz
*   Ambient Temperature = 25 degC
*   Inductor Saturation Model
*   Use model for Time Domain simulations
*======================================================================
.subckt XGL1010-682_sat port1 port2 PARAMS: Cpar=18.8pF Ind=6.8uH
X1 port1 port2 Model1C PARAMS:
+ R1=7
+ R2=0.0062
+ C= {Cpar}
+ K1=0.0003
+ K2=0.73
+ K3= {Ind}
+ K4=0.000001
+ K5=0.000001
+ L=7.0667E-06
+ Is=43.3005
+ a=0.7778
+ L_Z0=0
+ L_EL=0
+ L_F0=0E6
+ PkZ=2580.11473
.ends XGL1010-682_sat
```

In [14]:
def process_coilcraft_inductor_lib(filename: str):
    data_str = Path(filename).read_text() # Reads the file in a way that makes it able to be operated on like a string (and importantly maintains new lines)
    data = re.findall(r"(?s)(?=\*)(.*?)(?=\.SUBCKT)(.*?)(?<=\.ENDS)(.*?)(?=\n)", data_str, flags=re.S)  # This expression extracts each subckt 'clump'
    data = [''.join(x) for x in data]  # This takes all the groups that are generated in the above step and merges them into one string

    capacitors = []
    capacitance_regex = r"(?<=Property : C = )(.*?)(?=\[|\n)"
    part_number_regex = r"(?<=Murata P/N :)(.*?)(?=\n)"
    for inductor in data:
        try:
            capacitors.append({"PartNumber": re.findall(part_number_regex, inductor)[0], "Capacitance": float(re.findall(capacitance_regex, inductor)[0]) / 1000000, "SubCkt": inductor.replace('\n', ';')})  # The \n → ; will have to be reversed later but allows for it to be stored in a CSV, also capacitance is divided by 1000000 because it is in uF
        except:  # This occurs when a model is encountered which is not an inductor but one of the models required to represent an inductor
            pass

    return capacitors

### Initial Processing
This step generate the list of dicts of inductor data. Then it removes any duplicates (such as duplicate Model1A, Model1B, etc.) and saves the CSV. All model files have to be added manually to the schematic to make sure all inductors have the supporting model subckts.

In [15]:
capacitors = process_coilcraft_inductor_lib('GRM.lib')
capacitors_df = pd.DataFrame(capacitors).drop_duplicates()  # Convert the list of dicts to a pandas dataframe to make CSV-ifying easy
capacitors_df = capacitors_df.sort_values('Capacitance').reset_index(drop=True)
capacitors_df.to_csv('MurataCapacitors.csv') # Save it as the CSV file